In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


sys.path.append('../../src/')
from bonsai import load_bonsai_config
import clr
clr.AddReference("System")
import System

load_bonsai_config(r"C:\git\AllenNeuralDynamics\aind-vr-foraging\Bonsai")
import harp


In [2]:
HarpBehavior = harp.Device("Behavior")
AnalogInput = harp.Device("AnalogInput")

## Paths to the data:
root = Path(r'Z:\scratch\vr-foraging\scratchpad\pid_benchmarks\20230921')
rig = "VR2"
fname_formatter = lambda reg : f"reg__{reg}.bin"
# Harp.Behavior data:
harp_behavior_root = root / rig / "Behavior"

# Harp.AnalogInput data:
harp_analog_root = root / rig / "Analog"


In [10]:
odor_valve_onset = HarpBehavior.file_to_dataframe(harp_behavior_root / fname_formatter(34))
odor_valve_offset = HarpBehavior.file_to_dataframe(harp_behavior_root / fname_formatter(35))

digital_outputs = (HarpBehavior.module.DigitalOutputs)

pid_data = harp.read_harp_bin(harp_analog_root / fname_formatter(33))



In [57]:
digital_outputs = (HarpBehavior.module.DigitalOutputs)

available_io = [val.ToString() for val in System.Enum.GetValues(digital_outputs) if int(val) > 0]

for io in available_io:
    odor_valve_onset[io] = odor_valve_onset["Value"].apply(lambda entry: entry.HasFlag(System.Enum.Parse(digital_outputs, io)))
    odor_valve_onset

['DOPort0', 'DOPort1', 'DOPort2', 'SupplyPort0', 'SupplyPort1', 'SupplyPort2', 'Led0', 'Led1', 'Rgb0', 'Rgb1', 'DO0', 'DO1', 'DO2', 'DO3']


,Value,SupplyPort0
Seconds,,
190933.505504,SupplyPort1,False
190939.519488,SupplyPort2,False
190945.533504,SupplyPort1,False
190951.548480,SupplyPort2,False
190957.563488,SupplyPort1,False
190963.578496,SupplyPort2,False
190969.585504,SupplyPort1,False
190975.600480,SupplyPort2,False
190981.615488,SupplyPort1,False
